# Script prepare data square occupation
Script to create training image data for the CNN that classify an empty or occupied square.

For each single square create:
- Image named as "<dataset image no.>_<square coord>.png"
- Text file "<dataset image no.>_<square coord>.txt" containing the true label

In [1]:
import os, glob
import cv2
from FEN import FEN
from chessboard_detection import *

Change path according to where you have the dataset folder, default input dir

Change path according to where you prefer having the output

Regex:
- path/1** : start with '1' (1000 to 1999)
- path/** : all

rewrite: Set 'True' to rewrite old already processed images found in the output dir 

In [9]:
input_img_path = './../input/**'
dst_dir_path = './../output/training_squares/'
rewrite = False

In [10]:
if not os.path.exists(dst_dir_path):
    os.mkdir(dst_dir_path)

to_be_processed = glob.glob(input_img_path)
already_processed = []
if not rewrite:
    already_processed = glob.glob(f'{dst_dir_path}**')
    already_processed = list(set([os.path.splitext(filename)[0].split('\\')[-1].split('_')[0] for filename in already_processed]))

In [13]:
num_of_to_be_processed = len(to_be_processed)//2
num_of_already_processed = 0
last_progress_percentage_shown = 0

for in_process in to_be_processed:
    imgnumber = os.path.splitext(in_process)[0].split('\\')[-1]

    # percentage update
    progress_percentage = num_of_already_processed / num_of_to_be_processed * 100
    if progress_percentage - last_progress_percentage_shown > 10:
        last_progress_percentage_shown = progress_percentage
        print(f"########################### progress: {progress_percentage}% ###########################\n")

    #skip if not a file image, if json does not exists, if already present in output
    if not os.path.isfile(in_process):
        continue
    if not in_process.lower().endswith(".png"):
        continue
    print(f"{in_process}...", end=' ')
    if not os.path.isfile(os.path.splitext(in_process)[0] + '.json'):
        print(f"Not found related json({in_process})")
        continue
    if imgnumber in already_processed:
        num_of_already_processed += 1
        print(f"Already processed, skipped({in_process})")
        continue

    # load FEN true label
    truth = FEN(os.path.splitext(in_process)[0])
    true_fen, true_pos, viewpoint = truth.fen, truth.pieces, truth.view
    
    try:    
        # First pass preprocessing
        warpedBoardImg = board_detection(in_process, f"{'output_' + imgnumber}")
        if warpedBoardImg is None:
            num_of_already_processed += 1
            print("Skipped")
            continue

        # Second pass preprocessing
        grid_squares = grid_detection(warpedBoardImg, viewpoint)
        if grid_squares is None:
            print("Skipped")
            num_of_already_processed += 1
            continue
        
        # Extend the information to include piece information in 3rd col (image remain last in 4th col)
        grid_squares = np.column_stack((grid_squares[:,:2], 
                                            [true_pos.get(coord, 'empty') for coord in grid_squares[:, 1]],
                                            grid_squares[:,-2:]
                                            ))
        
        for square_coord, piece, square_img in grid_squares[:,1:4]:
            output_filename = f'{dst_dir_path}{imgnumber}_{square_coord}'
            
            # .png
            cv2.imwrite(output_filename + '.png', square_img)

            # .txt
            with open(f'{output_filename}.txt', 'w') as f:
                f.write(true_pos[square_coord] if square_coord in true_pos else 'empty')

        print('Done')
    except:
        output_filename = f'{dst_dir_path}{imgnumber}_error'
        with open(f'{output_filename}.txt', 'w') as f:
                f.write('Error somewhere')
        
    num_of_already_processed += 1
        

./../input\0000.png... Already processed, skipped(./../input\0000.png)
./../input\0001.png... Already processed, skipped(./../input\0001.png)
./../input\0024.png... Already processed, skipped(./../input\0024.png)
./../input\0026.png... Already processed, skipped(./../input\0026.png)
./../input\0039.png... Not enough lines found, provide a better image
Skipped
./../input\0060.png... Already processed, skipped(./../input\0060.png)
./../input\0081.png... Not enough lines found, provide a better image
Skipped
./../input\0132.png... Already processed, skipped(./../input\0132.png)
./../input\0192.png... Already processed, skipped(./../input\0192.png)
./../input\0195.png... Not enough lines found, provide a better image
Skipped
./../input\0206.png... Already processed, skipped(./../input\0206.png)
./../input\0231.png... Already processed, skipped(./../input\0231.png)
./../input\0280.png... Already processed, skipped(./../input\0280.png)
./../input\0292.png... Already processed, skipped(./../i

Clean square files created by image that raised errors

In [19]:
errors = glob.glob(f'{dst_dir_path}**error**')

errors = list(set([os.path.splitext(filename)[0].split('\\')[-1].split('_')[0] for filename in errors]))

for error in errors:
    for filename in glob.glob(f"{dst_dir_path}*{error}*"):
        if "error" in filename:
            continue
        print(f"Deleting {filename}")
        os.remove(filename)

Deleting ./../output/training_squares\4806_A1.png
Deleting ./../output/training_squares\4806_A1.txt
Deleting ./../output/training_squares\4806_A2.png
Deleting ./../output/training_squares\4806_A2.txt
Deleting ./../output/training_squares\4806_A3.png
Deleting ./../output/training_squares\4806_A3.txt
Deleting ./../output/training_squares\4806_A4.png
Deleting ./../output/training_squares\4806_A4.txt
Deleting ./../output/training_squares\4806_A5.png
Deleting ./../output/training_squares\4806_A5.txt
Deleting ./../output/training_squares\4806_A6.png
Deleting ./../output/training_squares\4806_A6.txt
Deleting ./../output/training_squares\4806_A7.png
Deleting ./../output/training_squares\4806_A7.txt
Deleting ./../output/training_squares\4806_B1.png
Deleting ./../output/training_squares\4806_B1.txt
Deleting ./../output/training_squares\4806_B2.png
Deleting ./../output/training_squares\4806_B2.txt
Deleting ./../output/training_squares\4806_B3.png
Deleting ./../output/training_squares\4806_B3.txt
